In [1]:

from tkinter import *
import requests
import random
import json
from datetime import datetime, timedelta, date

root = Tk()
root.title("Movie Search")
root.geometry("1500x1500")
root.config(bg="white")  # Set the background color of the root window to blue

frames = Listbox(root, bg="light blue")  # Set the background color of the Listbox to blue
frames.pack(side=LEFT, fill=BOTH, expand=True)

scrollbar = Scrollbar(root, orient='vertical', command=frames.yview)
scrollbar.pack(side=RIGHT, fill=Y)

frames.config(yscrollcommand=scrollbar.set)

myLabel1 = Label(frames, text="How do you want to search for movies?", bg="white")  # Set the background color of the Label to blue
myLabel1.pack(anchor=S)

item_list = ["Browse movies in the last year", "Search by Keyword", "Discover"]
value = StringVar()
value.set("Select an option")
dropdown = OptionMenu(frames, value, *item_list)
dropdown.config(bg="white")  
dropdown.pack(anchor=S)
frames.insert(END, "\n")
frames.insert(END, "\n")
frames.insert(END, "\n")
frames.insert(END, "\n")

def Select():
    selection = value.get()
    if selection == "Browse movies in the last year":
        clear_frames()
        releaseDate()
    elif selection == "Search by Keyword":
        clear_frames()
        searchByKeyword()
    elif selection == "Discover":
        clear_frames()
        discoverMovies()
        
def clear_frames():
    frames.delete(0, END)

def movie_info(results):
    clear_frames()
    for item in results:
        date = item['release_date']
        desc = item["overview"]
        title = item["original_title"]
        lang = item["original_language"]
        rating = item['vote_average']
        if lang == "en":
            frames.insert(END, f"{title}– {date}")
            frames.insert(END, f"Language: {lang}")
            frames.insert(END, f"Average Rating – {rating}")
            frames.insert(END, desc)
            frames.insert(END, "")

    
def releaseDate():
    api_key = "2724c22c82c5ef7e18c1dabc7f15b845"
    current_date = date.today()
    one_year_ago_today = datetime.now() - timedelta(days=365)
    from_release_date = one_year_ago_today.date()
    to_release_date = current_date
    raw_response = requests.get(f"https://api.themoviedb.org/3/discover/movie?primary_release_date.gte={from_release_date}&primary_release_date.lte={to_release_date}&api_key={api_key}").text
    response = json.loads(raw_response)
    results = response['results']
    movie_info(results)
    



def searchByKeyword():
    def searchMovies():
        keyword = entry.get()
        api_key = "2724c22c82c5ef7e18c1dabc7f15b845"
        raw_response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={keyword}").text
        response = json.loads(raw_response)
        results = response['results']
        movie_info(results)
        entry.delete(0, END)
    entry = Entry(frames)
    entry.pack(anchor=S)

    search_button = Button(frames, text="Search movies", command=searchMovies)
    search_button.pack(anchor=S)



def discoverMovies():
    api_key = "2724c22c82c5ef7e18c1dabc7f15b845"
    total_pages = 3
    for page in range(1, total_pages + 1):
        raw_response = requests.get(f"https://api.themoviedb.org/3/movie/top_rated?api_key={api_key}&language=en-US&page={page}").text
        response = json.loads(raw_response)
        results = response['results']
        movie_info(results)



choose_button = Button(frames, text="Click to choose", command=Select)
choose_button.pack(anchor=S)

root.mainloop()